[View in Colaboratory](https://colab.research.google.com/github/tcglarry/aoi/blob/master/aoi_inceptionresnet_save_model_output_2.ipynb)

In [1]:
# Install a Drive FUSE wrapper.
# https://github.com/astrada/google-drive-ocamlfuse
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

Preconfiguring packages ...
Selecting previously unselected package cron.
(Reading database ... 18408 files and directories currently installed.)
Preparing to unpack .../00-cron_3.0pl1-128ubuntu5_amd64.deb ...
Unpacking cron (3.0pl1-128ubuntu5) ...
Selecting previously unselected package libapparmor1:amd64.
Preparing to unpack .../01-libapparmor1_2.11.0-2ubuntu17.1_amd64.deb ...
Unpacking libapparmor1:amd64 (2.11.0-2ubuntu17.1) ...
Selecting previously unselected package libdbus-1-3:amd64.
Preparing to unpack .../02-libdbus-1-3_1.10.22-1ubuntu1_amd64.deb ...
Unpacking libdbus-1-3:amd64 (1.10.22-1ubuntu1) ...
Selecting previously unselected package dbus.
Preparing to unpack .../03-dbus_1.10.22-1ubuntu1_amd64.deb ...
Unpacking dbus (1.10.22-1ubuntu1) ...
Selecting previously unselected package dirmngr.
Preparing to unpack .../04-dirmngr_2.1.15-1ubuntu8.1_amd64.deb ...
Unpacking dirmngr (2.1.15-1ubuntu8.1) ...
Selecting previously unselected package distro-info-data.
Preparing to unpack .

In [0]:
# Generate auth tokens for Colab
from google.colab import auth
auth.authenticate_user()

In [4]:
# Generate creds for the Drive FUSE library.
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [5]:
!mkdir -p drive
!google-drive-ocamlfuse drive
!ls drive

aoi_aoi_test1.ipynb			 Untitled0.ipynb
aoi_test				 Untitled1.ipynb
Colab Notebooks				 Untitled2.ipynb
PiXNet					 Untitled3.ipynb
Sample upload.txt			 Untitled4.ipynb
TAHU PowerPoint Template.pdf		 Untitled5.ipynb
TAHU PowerPoint Template - 柱狀圖 1.ods  Untitled6.ipynb
test for excel				 Untitled7.ipynb


In [0]:
!cp drive/aoi_test/train_images.zip .

In [8]:
!unzip train_images.zip

Archive:  train_images.zip
   creating: train_images/
 extracting: train_images/train_00000.png  
 extracting: train_images/train_00001.png  
 extracting: train_images/train_00002.png  
 extracting: train_images/train_00003.png  
 extracting: train_images/train_00004.png  
 extracting: train_images/train_00005.png  
 extracting: train_images/train_00006.png  
 extracting: train_images/train_00007.png  
 extracting: train_images/train_00008.png  
 extracting: train_images/train_00009.png  
 extracting: train_images/train_00010.png  
 extracting: train_images/train_00011.png  
 extracting: train_images/train_00012.png  
 extracting: train_images/train_00013.png  
 extracting: train_images/train_00014.png  
 extracting: train_images/train_00015.png  
 extracting: train_images/train_00016.png  
 extracting: train_images/train_00017.png  
 extracting: train_images/train_00018.png  
 extracting: train_images/train_00019.png  
 extracting: train_images/train_00020.png  
 extracting: train_ima

In [1]:
from keras.applications.inception_v3 import InceptionV3
from keras.applications.densenet import DenseNet121
from keras.preprocessing import image

from keras.applications.inception_resnet_v2 import InceptionResNetV2

from keras.applications.mobilenet import MobileNet


from keras.layers import Dense, GlobalAveragePooling2D
import time

import cv2
import numpy as np
import matplotlib.pyplot as plt
import keras.backend as K
import tensorflow as tf
import subprocess
import os
import pickle
from keras.layers import Input, Conv2D, MaxPooling2D, merge, Lambda,UpSampling2D, concatenate, Reshape,Cropping2D
from keras.models import Model, load_model
import pandas as pd
import sklearn 
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint
from keras.callbacks import Callback

Using TensorFlow backend.


In [0]:
src_1 = 'drive/aoi_test/'
src_2= ''

In [3]:
train_df = pd.read_csv('train.csv')


train_img =  train_df['ID'].values

print ('iamge  size', train_img.shape)
img_list=[]
for i,img in enumerate(train_img):
    x = cv2.imread(src_2+'train_images/'+img)
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    #x = x/127.5 -1.
    img_list.append(x)
    
print ('done')

iamge  size (2528,)
done


In [4]:
train_data = np.array(img_list)
print (train_data.shape)
#train_data =  train_data[:,:,:,np.newaxis]
print (train_data.shape)

(2528, 512, 512, 3)
(2528, 512, 512, 3)


here


In [10]:


inputs = Input(shape=(512,512,3))
inputs = Lambda(lambda x: x/275. - 1.0)(inputs)
#pool0 = x = MaxPooling2D(pool_size=(2,2))(inputs)

#inputs_mobnet = Cropping2D(cropping=16,data_format = "channels_last")(pool0)
print ('inputs_mobnet', inputs.get_shape())
# create the base pre-trained model
#base_model = MobileNet(weights='imagenet',input_tensor=inputs_mobnet,input_shape=(224,224,3), include_top=False)

# create the base pre-trained model
base_model = InceptionResNetV2(weights='imagenet',input_tensor=inputs, input_shape=(512,512,3), include_top=False)
x = base_model.output
print (x.get_shape)
model = Model(inputs=base_model.input, outputs=x)
model.summary()

inputs_mobnet (?, 512, 512, 3)
<bound method Tensor.get_shape of <tf.Tensor 'conv_7b_ac/Relu:0' shape=(?, 14, 14, 1536) dtype=float32>>
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 512, 512, 3)  0                                            
__________________________________________________________________________________________________
lambda_3 (Lambda)               (None, 512, 512, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 255, 255, 32) 864         lambda_3[0][0]                   
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 255, 255, 32) 96          conv2d_

In [0]:

def build_model():
    inputs = Input(shape=(512,512,3))
    inputs = Lambda(lambda x: x/275. - 1.0)(inputs)
    # create the base pre-trained model
    base_model = InceptionResNetV2(weights='imagenet',input_tensor=inputs, input_shape=(512,512,3), include_top=False)
    x = base_model.output
    print (x.get_shape)
    model = Model(inputs=base_model.input, outputs=x)
    model.summary()

    return model


In [6]:
model_3 = build_model()
# compile the model (should be done *after* setting layers to non-trainable)
model_3.compile(optimizer='Nadam', loss='categorical_crossentropy',metrics=['accuracy'])

<bound method Tensor.get_shape of <tf.Tensor 'conv_7b_ac/Relu:0' shape=(?, 14, 14, 1536) dtype=float32>>
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 512, 512, 3)  0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 512, 512, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 255, 255, 32) 864         lambda_1[0][0]                   
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 255, 255, 32) 96          conv2d_1[0][0]                   
____

In [7]:
s = time.time()
print ('start prediction')
prediction = model_3.predict(train_data)
print (prediction.shape)
e = time.time()
print ('prediction time', round(e - s, 4))

start prediction
(2528, 14, 14, 1536)
prediction time 227.3008


In [8]:
s = time.time()
np.save(src_1+'inceptionresnet_3.npy',prediction)
e = time.time()
print ('model output saved, total time=', round(e - s, 4))

model output saved, total time= 448.3113
